<a href="https://colab.research.google.com/github/changhorang/SSAC_study/blob/main/DL3%20(PyTorch)/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문장을 벡터로 변환하기_BERT

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 2.8 MB 12.2 MB/s 
     |████████████████████████████████| 806 kB 47.1 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 53.9 MB/s 
     |████████████████████████████████| 829 kB 53.7 MB/s 
     |████████████████████████████████| 332 kB 54.4 MB/s 
     |████████████████████████████████| 636 kB 58.6 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 3.3 MB 51.5 MB/s 
     |████████████████████████████████| 61 kB 443 kB/s 
     |████████████████████████████████| 1.1 MB 57.0 MB/s 
     |████████████████████████████████| 96 kB 4.7 MB/s 
     |████████████████████████████████| 192 kB 40.6 MB/s 
     |████████████████████████████████| 160 kB 50.0 MB/s 
     |████████████████████████████████| 271 kB 62.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

## tokenizer

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('beomi/kcbert-base', do_lower_case=False)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

## Model initialization
- BERT 모델이 pretrain할 때 사용한 토크나이저를 그대로 이용해야 벡터 변환에 문제가 없음! (model과 tokenizer의 토큰화 방식 동일하게)



In [3]:
from transformers import BertConfig, BertModel

pretrained_model_config = BertConfig.from_pretrained('beomi/kcbert-base')

model = BertModel.from_pretrained('beomi/kcbert-base', config=pretrained_model_config)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# BERT model pretrain시에 설정한 내용
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

## 입력값 만들기

In [7]:
sentences = ['안녕하세요', '헬로!', '하이!']

features = tokenizer(sentences, 
                     max_length=10, # 토큰 최대 길이
                     padding='max_length', # 토큰 최대 길이에 맞게 부족하면 padding(0) 채우기
                     truncation=True) # max_length보다 길면 문장 자르기

features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 3424, 4091, 5, 3, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

## BERT로 단어/문장 수준 벡터 구하기

In [9]:
# features: torch.Tensor로 변환
import torch

features = {k: torch.tensor(v) for k, v in features.items()}
features

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


{'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[    2, 19017,  8482,     3,     0,     0,     0,     0,     0,     0],
         [    2,  3424,  4091,     5,     3,     0,     0,     0,     0,     0],
         [    2, 15830,     5,     3,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [13]:
# embedding 계산하기
outputs = model(**features)
outputs

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
                                                        [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
                                                        [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
                                                        ...,
                                                        [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
                                                        [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
                                                        [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],
                                               
                                                       [[ 0.6264, -0.8184,  1.3362,  ..., -0.1480, -0.3773,  0.677

In [14]:
# outputs.last_hidden_state.shape
outputs[0].shape # [3, 10, 768] = 문장수, 토큰최대길이, 임베딩dim

torch.Size([3, 10, 768])

In [15]:
# 각 문장의 토큰별 대응되는 벡터
# outputs.pooler_output: h1~768(CLS)벡터에 마지막 layer 곱한 뒤 tanh를 취한 결과 (출력 범위는 -1 ~ 1)
outputs[1]

tensor([[-1.5944e-01,  5.4692e-02,  1.1013e-01,  ...,  2.6839e-01,
          1.5962e-01, -9.8279e-01],
        [ 2.8155e-01,  6.3905e-04,  1.0069e-02,  ...,  4.9305e-01,
          6.1817e-02, -9.9000e-01],
        [-9.2214e-01,  2.9692e-01, -1.1048e-02,  ...,  4.2906e-01,
          3.1099e-02, -9.9551e-01]], grad_fn=<TanhBackward0>)

In [16]:
outputs[1].shape

torch.Size([3, 768])